
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.
import numpy as np
import pandas as pd
try:
    df = pd.read_csv('data/Corona_NLP_train.csv', encoding='utf-8')  # Change 'utf-8' to the correct encoding if needed
except UnicodeDecodeError:
    # If a UnicodeDecodeError occurs, try a different encoding
    df = pd.read_csv('data/Corona_NLP_train.csv', encoding='latin1') 

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.
df = df[df['Sentiment'] != 'Neutral']
X = df['OriginalTweet']
numeric_var = {'Sentiment': {'Positive':1, 'Extremely Positive':1, 'Negative':0, 'Extremely Negative':0}}
df = df.replace(numeric_var)
Y = df['Sentiment']
total_samples = len(df)

# Calculate indices for splitting
train_end = int(0.6 * total_samples)  # 60% for training
test_end = train_end + int(0.15 * total_samples)  # 15% for testing

# Split the DataFrame
train_df = X.iloc[:train_end]
test_df = X.iloc[train_end:test_end]
validation_df = X.iloc[test_end:]
tr_y = Y.iloc[:train_end]
te_y = Y.iloc[train_end:test_end]
val_y = Y.iloc[test_end:]
# Make sure that all your data is numpy arrays and not pandas dataframes or series.
X = np.array(X)
Y = np.array(Y)

X_train = np.array(train_df)
Y_train = np.array(tr_y)
X_test = np.array(test_df)
Y_test = np.array(te_y)
X_valid = np.array(validation_df)
Y_valid = np.array(val_y)

In [2]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('vectorizer', CountVectorizer()),  # Step 1: Convert text to numerical features
    ('classifier', LogisticRegression(max_iter=1000, solver='lbfgs', random_state=42))  # Step 2: Logistic regression
])

In [3]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.
from sklearn.metrics import precision_score, recall_score
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)



precision_0 = precision_score(Y_test, y_pred, pos_label=0)
precision_1 = precision_score(Y_test, y_pred, pos_label=1)
recall_0 = recall_score(Y_test, y_pred, pos_label=0)
recall_1 = recall_score(Y_test, y_pred, pos_label=1)

In [4]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    if not hasattr(model, "predict_proba"):
        raise ValueError("The model must have a predict_proba method.")

    probabilities = model.predict_proba(X)[:, 1]

    predictions = (probabilities >= threshold).astype(int)

    costs = []
    for true_label, pred_label in zip(Y, predictions):
        if true_label == 1 and pred_label == 0: 
            costs.append(1)
        elif true_label == 0 and pred_label == 1:
            costs.append(5)
        else: 
            costs.append(0)

    return sum(costs) / len(costs)

In [5]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.
import numpy as np
def find_optimal_threshold(model, X_test, Y_test):
    thresholds = np.linspace(0, 1, 101)  
    costs = [cost(model, threshold, X_test, Y_test) for threshold in thresholds]
    optimal_index = np.argmin(costs)
    return thresholds[optimal_index], costs[optimal_index]

optimal_threshold, cost_at_optimal_threshold = find_optimal_threshold(model, X_test, Y_test)

In [6]:

# Part 6

def compute_cost_and_interval(model, threshold, X_valid, Y_valid, confidence=0.99):
    cost_at_optimal_threshold_valid = cost(model, threshold, X_valid, Y_valid)

    n = len(Y_valid)
    max_cost_value = threshold  # Adjust this if your cost function has a different max value
    min_cost_value = 0  # Assuming cost cannot be negative
    epsilon = (max_cost_value - min_cost_value) / 2  # Half the range as epsilon

    # Calculate the bound using Hoeffding's inequality
    bound = np.sqrt((np.log(2 / (1 - confidence))) / (2 * n))
    
    # Calculate the confidence interval
    lower_bound = cost_at_optimal_threshold_valid - bound * epsilon
    upper_bound = cost_at_optimal_threshold_valid + bound * epsilon
    
    return cost_at_optimal_threshold_valid, (lower_bound, upper_bound)

cost_at_optimal_threshold_valid, cost_interval_valid = compute_cost_and_interval(model, optimal_threshold, X_valid, Y_valid)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

In [7]:

# Part 7

def compute_C(model, threshold, X_valid, Y_valid):
    probabilities = model.predict_proba(X_valid)[:, 1]
    predictions = (probabilities >= threshold).astype(int)
    
    C = [(1 - (pred == 1)) * true + 5 * (1 - true) * (pred == 1)
         for true, pred in zip(Y_valid, predictions)]
    
    return np.array(C)

C_valid = compute_C(model, optimal_threshold, X_valid, Y_valid)

variance_of_C = np.var(C_valid)

confidence = 0.99
n = len(C_valid)
epsilon_bennett = np.sqrt((2 * np.log(2 / (1 - confidence)) * variance_of_C) / n)

mean_C = np.mean(C_valid)

lower_bound_bennett = mean_C - epsilon_bennett
upper_bound_bennett = mean_C + epsilon_bennett

interval_of_C = (lower_bound_bennett, upper_bound_bennett)

variance_of_C, interval_of_C
assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

In [8]:
# ASSIGNMENT 4, TEST 1, POINTS 24

Numerical error -3.932618120039422e-15
Numerical error -3.932618120039422e-15
Beginning tests for problem 1

---------------------------------
Beginning test for part1
---------------------------------

-----Beginning test------
X.shape is correct, it is (33444,)
-----Ending test---------

-----Beginning test------
Y.shape is correct, it is (33444,)
-----Ending test---------

-----Beginning test------
X seems to be a list of strings
-----Ending test---------

-----Beginning test------
Y contains the correct values {0,1}
-----Ending test---------

-----Beginning test------
Shape of X_train correct it is (20066,)
-----Ending test---------

-----Beginning test------
Shape of X_test correct it is (5016,)
-----Ending test---------

-----Beginning test------
Shape of X_valid correct it is (8362,)
-----Ending test---------

-----Beginning test------
Shape of Y_train correct it is (20066,)
-----Ending test---------

-----Beginning test------
Shape of Y_test correct it is (5016,)
-----Ending te

The number of points you have scored in total for this entire set of Problems is 22 out of 24.